In [ ]:
import math
import numpy as np
import string
import json
import re
# from multiprocessing import Pool, cpu_count
import string
import re
from gensim.parsing.preprocessing import remove_stopwords
import time
!pip install underthesea

     |████████████████████████████████| 7.6 MB 6.0 MB/s 
     |████████████████████████████████| 235 kB 45.6 MB/s 
     |████████████████████████████████| 581 kB 58.0 MB/s 
     |████████████████████████████████| 743 kB 58.6 MB/s 


# BM25 algorithms

In [ ]:
# BM25 algorithms from rank_bm25

class BM25:
    def __init__(self, corpus, tokenizer=None):
        self.corpus_size = len(corpus)
        self.avgdl = 0
        self.doc_freqs = []
        self.idf = {}
        self.doc_len = []
        self.tokenizer = tokenizer

        if tokenizer:
            corpus = self._tokenize_corpus(corpus)

        nd = self._initialize(corpus)
        self._calc_idf(nd)

    def _initialize(self, corpus):
        nd = {}  # word -> number of documents with word
        num_doc = 0
        for document in corpus:
            self.doc_len.append(len(document))
            num_doc += len(document)

            frequencies = {}
            for word in document:
                if word not in frequencies:
                    frequencies[word] = 0
                frequencies[word] += 1
            self.doc_freqs.append(frequencies)

            for word, freq in frequencies.items():
                try:
                    nd[word]+=1
                except KeyError:
                    nd[word] = 1

        self.avgdl = num_doc / self.corpus_size
        return nd

    def _tokenize_corpus(self, corpus):
        pool = Pool(cpu_count())
        tokenized_corpus = pool.map(self.tokenizer, corpus)
        return tokenized_corpus

    def _calc_idf(self, nd):
        raise NotImplementedError()

    def get_scores(self, query):
        raise NotImplementedError()

    def get_batch_scores(self, query, doc_ids):
        raise NotImplementedError()

    def get_top_n(self, query, documents, n=5):
        assert self.corpus_size == len(documents)
        scores = self.get_scores(query)
        top_n = np.argsort(scores)[::-1][:n]
        return [documents[i] for i in top_n]


class BM25Okapi(BM25):
    def __init__(self, corpus, tokenizer=None, k1=1.5, b=0.75, epsilon=0.25):
        self.k1 = k1
        self.b = b
        self.epsilon = epsilon
        super().__init__(corpus, tokenizer)

    def _calc_idf(self, nd):
        idf_sum = 0
        negative_idfs = []
        for word, freq in nd.items():
            idf = math.log(self.corpus_size - freq + 0.5) - math.log(freq + 0.5)
            self.idf[word] = idf
            idf_sum += idf
            if idf < 0:
                negative_idfs.append(word)
        self.average_idf = idf_sum / len(self.idf)

        eps = self.epsilon * self.average_idf
        for word in negative_idfs:
            self.idf[word] = eps

    def get_scores(self, query):
        score = np.zeros(self.corpus_size)
        doc_len = np.array(self.doc_len)
        for q in query:
            q_freq = np.array([(doc.get(q) or 0) for doc in self.doc_freqs])
            score += (self.idf.get(q) or 0) * (q_freq * (self.k1 + 1) /
                                               (q_freq + self.k1 * (1 - self.b + self.b * doc_len / self.avgdl)))
        return score

class BM25Robertson(BM25):
    def __init__(self, corpus, tokenizer=None, k1=1.5, b=0.75, epsilon=0.25):
        self.k1 = k1
        self.b = b
        self.epsilon = epsilon
        super().__init__(corpus, tokenizer)

    def _calc_idf(self, nd):
        idf_sum = 0
        negative_idfs = []
        for word, freq in nd.items():
            idf = math.log(self.corpus_size - freq + 0.5) - math.log(freq + 0.5)
            self.idf[word] = idf
            idf_sum += idf
            if idf < 0:
                negative_idfs.append(word)
        self.average_idf = idf_sum / len(self.idf)

        eps = self.epsilon * self.average_idf
        for word in negative_idfs:
            self.idf[word] = eps

    def get_scores(self, query):
        score = np.zeros(self.corpus_size)
        doc_len = np.array(self.doc_len)
        for q in query:
            q_freq = np.array([(doc.get(q) or 0) for doc in self.doc_freqs])
            score += (self.idf.get(q) or 0) * (q_freq /
                                               (q_freq + self.k1 * (1 - self.b + self.b * doc_len / self.avgdl)))
        return score

class BM25ATIRE(BM25):
    def __init__(self, corpus, tokenizer=None, k1=1.5, b=0.75, epsilon=0.25):
        self.k1 = k1
        self.b = b
        self.epsilon = epsilon
        super().__init__(corpus, tokenizer)

    def _calc_idf(self, nd):
        idf_sum = 0
        negative_idfs = []
        for word, freq in nd.items():
            idf = math.log(self.corpus_size) - math.log(freq)
            self.idf[word] = idf
            idf_sum += idf
            if idf < 0:
                negative_idfs.append(word)
        self.average_idf = idf_sum / len(self.idf)

        eps = self.epsilon * self.average_idf
        for word in negative_idfs:
            self.idf[word] = eps

    def get_scores(self, query):
        score = np.zeros(self.corpus_size)
        doc_len = np.array(self.doc_len)
        for q in query:
            q_freq = np.array([(doc.get(q) or 0) for doc in self.doc_freqs])
            score += (self.idf.get(q) or 0) * (q_freq * (self.k1 + 1) /
                                               (q_freq + self.k1 * (1 - self.b + self.b * doc_len / self.avgdl)))
        return score

class BM25L(BM25):
    def __init__(self, corpus, tokenizer=None, k1=1.5, b=0.75, delta=0.5):
        # Algorithm specific parameters
        self.k1 = k1
        self.b = b
        self.delta = delta
        super().__init__(corpus, tokenizer)

    def _calc_idf(self, nd):
        for word, freq in nd.items():
            idf = math.log(self.corpus_size + 1) - math.log(freq + 0.5)
            self.idf[word] = idf

    def get_scores(self, query):
        score = np.zeros(self.corpus_size)
        doc_len = np.array(self.doc_len)
        for q in query:
            q_freq = np.array([(doc.get(q) or 0) for doc in self.doc_freqs])
            ctd = q_freq / (1 - self.b + self.b * doc_len / self.avgdl)
            score += (self.idf.get(q) or 0) * q_freq * (self.k1 + 1) * (ctd + self.delta) / \
                     (self.k1 + ctd + self.delta)
        return score

class BM25Plus(BM25):
    def __init__(self, corpus, tokenizer=None, k1=1.5, b=0.75, delta=1):
        # Algorithm specific parameters
        self.k1 = k1
        self.b = b
        self.delta = delta
        super().__init__(corpus, tokenizer)

    def _calc_idf(self, nd):
        for word, freq in nd.items():
            idf = math.log((self.corpus_size + 1) / freq)
            self.idf[word] = idf

    def get_scores(self, query):
        score = np.zeros(self.corpus_size)
        doc_len = np.array(self.doc_len)
        for q in query:
            q_freq = np.array([(doc.get(q) or 0) for doc in self.doc_freqs])
            score += (self.idf.get(q) or 0) * (self.delta + (q_freq * (self.k1 + 1)) /
                                               (self.k1 * (1 - self.b + self.b * doc_len / self.avgdl) + q_freq))
        return score

# Upload dataset
link: https://www.kaggle.com/yuiikin/vietnamese-vnexpress-news

In [ ]:
#upload texts.json and create corpuses
from google.colab import files
uploaded=files.upload() 

Saving health_news.csv to health_news.csv


In [ ]:
import pandas as pd

df = pd.read_csv('health_news.csv')
data = df['content']

docs = data.values.tolist()

In [ ]:
from underthesea import word_tokenize

corpus = [doc.translate(str.maketrans('','', string.punctuation)) for doc in docs]
corpus = [word_tokenize(sentence) for sentence in corpus]

In [ ]:
lower_cased_corpus = []
for sentence in corpus:
  sentence = [word.lower() for word in sentence]
  lower_cased_corpus.append(sentence)

# Stopwords processing (maybe unnecessary)

In [ ]:
# vietnamese stopwords
import gensim

# prepare a empty frozenset for vietnamese stopwords
all_stopwords = gensim.parsing.preprocessing.STOPWORDS
all_stopwords = all_stopwords.difference(all_stopwords)

# upload vietnamese stopwords
vietnamese_stopwords_set=[]
uploaded=files.upload()
sample = open("vietnamese-stopwords.txt","r")
for i, l in enumerate(sample):
  l = l.replace('\n', '')
  vietnamese_stopwords_set.append(l)

# add vietnamese stopwords to frozenset
stopwords = all_stopwords.union(set(vietnamese_stopwords_set))

In [ ]:
lower_cased_corpus_without_sw = []
for doc in lower_cased_corpus:
  doc = [word for word in doc if not word in stopwords]
  lower_cased_corpus_without_sw.append(doc)

# Corpus indexing

In [ ]:
bm25okapi = BM25Okapi(lower_cased_corpus)
bm25robertson = BM25Robertson(lower_cased_corpus)
bm25atire = BM25ATIRE(lower_cased_corpus)
bm25l = BM25L(lower_cased_corpus)
bm25plus = BM25Plus(lower_cased_corpus)

# Test

In [ ]:
query = "Làm sao để giảm cân?"
print("Query:")
print(query, "\n")

query = query.translate(str.maketrans('', '', string.punctuation))
query = word_tokenize(query)
lower_cased_query = [word.lower() for word in query]

result_docs = bm25l.get_top_n(lower_cased_query, docs, n = 10)
print("Document:")

print(result_docs[0])

Query:
Làm sao để giảm cân? 

Document:
Những chiếc váy với thiết kế bó sát cơ thể làm tôn lên thân hình thon thả và số đo 3 vòng 92-67-89 cm. Mở chiếc điện thoại, cô gái nở nụ cười tươi và tạo hình nhiều tư thế khác nhau. Trước đây do thân hình mũm mĩm, Linh chỉ mặc quần áo tối màu vì như vậy nhìn cơ thể sẽ gầy hơn. Tuy nhiên giờ đây, cô có thể tự do mặc những chiếc váy mình thích mà không còn cảm thấy tự ti như trước.
"Từ rất lâu rồi, mình ao ước có được thân hình thon gọn để mặc váy xinh giống các bạn. Cuối cùng thì ngày này đã đến", Linh nói. Ước mơ tưởng chừng có gì to tát nhưng lại là điều đeo bám trong cô gái nhiều năm.
Cất những bộ váy vào tủ, Linh lấy ra những bức hình hồi học phổ thông. Bức ảnh mặc áo dài ngày bế giảng năm lớp 12 là một minh chứng rõ nét cho thể hình mũm mĩm, thừa cân. Đây là lúc cân nặng của cô đạt đỉnh điểm 64 kg, trong khi chiều cao khiêm tốn 1,55 m. Dù cố gắng trang điểm xinh đẹp nhưng cô gái vẫn không thể tự tin khi chụp ảnh cùng các bạn.
"Béo cũng đáng 